# Mixer correction

Setup
-----

First, we are going to import the required packages.

In [52]:
# Import ipython widgets
import json
import math
import os

import ipywidgets as widgets
import matplotlib.pyplot
import numpy as np
import rich
import pandas as pd

# Set up the environment.
import scipy.signal
from IPython.display import display
from ipywidgets import fixed, interact, interact_manual, interactive

from qblox_instruments import Cluster, PlugAndPlay, Pulsar
from IPython.display import display, HTML
display(HTML("<style>.container { width:100% !important; }</style>"))

In [53]:
# This is the table that we want to populate
calibration = pd.DataFrame()
calibration["module"]  = []
calibration["complex_output"] = []
calibration["sequencer index"] = []
calibration["lo_freq (Hz)"] = []
calibration["if (Hz)"] = []
calibration["dc_mixer_offset_I"] = []
calibration["dc_mixer_offset_Q"] = []
calibration["mixer_amp_ratio"] = []
calibration["mixer_phase_error_deg"] = []
calibration

,module,complex_output,sequencer index,lo_freq (Hz),if (Hz),dc_mixer_offset_I,dc_mixer_offset_Q,mixer_amp_ratio,mixer_phase_error_deg


## Create Q1ASM program to play on the device

In [54]:
#Waveform dictionary (data will hold the samples and index will be used to select the waveforms in the instrument).
waveforms = {
             "I":     {"data": [], "index": 0},
             "Q":     {"data": [], "index": 1}
            }

waveforms["I"]["data"] = [0.4]*8000
waveforms["Q"]["data"] = [0.0]*8000

#Sequence program.
prog = f"start: set_mrk 15 \n play 0,1,1000\n set_mrk 15 \n play 0,1,1000\njmp @start"

#Sequence program which stops.
# prog = """
#         move 1250000, R0
#         set_mrk    15
# start:
#         reset_ph
#         upd_param  4
#         play       0,1,4
#         wait       7996
#         loop       R0,@start
#         set_mrk    0
#         upd_param  4
#         stop
# """

#Reformat waveforms to lists if necessary.
for name in waveforms:
    if str(type(waveforms[name]["data"]).__name__) == "ndarray":
        waveforms[name]["data"] = waveforms[name]["data"].tolist()  # JSON only supports lists

#Add sequence program and waveforms to single dictionary and write to JSON file.
wave_and_prog_dict = {"waveforms": waveforms, "weights": {}, "acquisitions": {}, "program": prog}
with open("sequence.json", 'w', encoding='utf-8') as file:
    json.dump(wave_and_prog_dict, file, indent=4)
    file.close()

## Select Device

We scan for the available devices connected via ethernet using the Plug & Play functionality of the Qblox Instruments package (see [Plug & Play](https://qblox-qblox-instruments.readthedocs-hosted.com/en/master/api_reference/pnp.html) for more info).

In [4]:
# Scan for available devices and display
# with PlugAndPlay() as p:
#     # get info of all devices
#     device_list = p.list_devices()
#     device_keys = list(device_list.keys())

# # rich.print(device_list)
# # create widget for names and ip addresses
# connect = widgets.Dropdown(
#     options=[(device_list[key]["identity"]["ip"]) for key in device_list.keys()],
#     description="Select Device",
# )
# display(connect)

Dropdown(description='Select Device', options=(), value=None)

In [55]:
# close all previous connections to the cluster
Cluster.close_all()

#device_name = "pingu_cluster"
device_name = "loki_cluster"
# ip_address = connect.v./alue
ip_address = '192.0.2.72'
# connect to the cluster and reset
cluster = Cluster(device_name, ip_address)
cluster.reset()
print(f"{device_name} connected at {ip_address}")
# cluster.identify()

loki_cluster connected at 192.0.2.72


## Select Module in Device

In [153]:
# Find all QRM/QCM modules
cluster.reset()
available_slots = {}
for module in cluster.modules:
    # if module is currently present in stack
    if cluster._get_modules_present(module.slot_idx):
        # check if QxM is RF or baseband
        if module.is_rf_type:
            available_slots[f"module{module.slot_idx}"] = ["QCM-RF", "QRM-RF"][
                module.is_qrm_type
            ]
        else:
            available_slots[f"module{module.slot_idx}"] = ["QCM", "QRM"][
                module.is_qrm_type
            ]

# List of all QxM modules present
connect_qxm = widgets.Dropdown(options=[key for key in available_slots.keys()])
display(connect_qxm)

Dropdown(options=('module1', 'module2', 'module3', 'module4', 'module5', 'module6', 'module7', 'module8', 'mod…

In [193]:
# Connect to the cluster QxM module
module = connect_qxm.value
print(module)
qxm = getattr(cluster, module)
print(f"{available_slots[connect_qxm.value]} connected")
print(cluster.get_system_state())

module17
QRM-RF connected
Status: CRITICAL, Flags: TEMPERATURE_OUT_OF_RANGE, Slot flags: NONE


In [194]:
qxm.out0_att(0)

## Select complex output

In [195]:
outputs = set(map(lambda i: i.split("_")[0], filter(lambda k: "out" in k and (not k.startswith("_")), dir(qxm))))
select_out = widgets.Dropdown(options = [key for key in outputs])
display(select_out)

Dropdown(options=('out0', 'disconnect'), value='out0')

In [196]:
qubits = ['q16','q17','q18','q19','q20','q21','q22','q23','q24','q25']
lo_list = [4.25025000e+09, 5.41650000e+09, 4.83750000e+09, 5.14600000e+09,
       4.40025000e+09, 5.21475000e+09, 4.72275000e+09, 5.27716667e+09,
       4.63350000e+09, 5.31275000e+09]
qubit_lo = dict(zip(qubits, lo_list))
qubit_lo

{'q16': 4250250000.0,
 'q17': 5416500000.0,
 'q18': 4837500000.0,
 'q19': 5146000000.0,
 'q20': 4400250000.0,
 'q21': 5214750000.0,
 'q22': 4722750000.0,
 'q23': 5277166670.0,
 'q24': 4633500000.0,
 'q25': 5312750000.0}

In [197]:
# lo_frequency = float(input(f"Enter desired LO frequency (Hz) for '{select_out.value}': "))
# lo_frequency = qubit_lo['q25']
# lo_frequency = 6793000000
lo_frequency = 6804000000
if qxm.module_type.name == "QRM":
    # qrms only have one complex output
    qxm.out0_in0_lo_en(True)
    qxm.out0_in0_lo_freq(lo_frequency)
else:
    lo_en = getattr(qxm, f"{select_out.value}_lo_en")
    lo_freq = getattr(qxm, f"{select_out.value}_lo_freq")
    lo_en(True)
    lo_freq(lo_frequency)
print(lo_frequency)

6804000000


## Select Sequencer in Module

In [198]:
qxm.stop_sequencer()
sequencers = { k : getattr(qxm, k) for k in filter(lambda a: a.startswith("sequencer") and (a != "sequencers"), dir(qxm)) }
select_seq = widgets.Dropdown(options = [key for key in sequencers.keys()])
display(select_seq)

Dropdown(options=('sequencer0', 'sequencer1', 'sequencer2', 'sequencer3', 'sequencer4', 'sequencer5'), value='…

## Upload program to sequencer

In [199]:
seq = sequencers[select_seq.value]
# cz = 500
# MY_IF = (500-cz)*1e6
MY_IF = -100e6
seq.nco_freq(MY_IF)
# seq.nco_freq(MY_IF := float(input("ENTER IF: ")))   # using 10 MHz IF

In [200]:
seq.sequence(os.path.join(os.getcwd(), "sequence.json"))
seq.mod_en_awg(True)
print(MY_IF)
print(seq.name)

-100000000.0
loki_cluster_module17_sequencer0


## Control sliders

In [201]:
rich.print(f"1.\tConnect complex output '{select_out.value}' of cluster '{device_name}' module '{module}' to a spectrum analyser.")
rich.print(f"2.\tIn the spectrum analyser, set the center frequency to {lo_frequency/1e9} GHz")
rich.print(f"3.\tAdjust the gain ratio and phase offset sliders until the LSB is gone.")
rich.print(f"4.\tAdjust the I and Q offsets until the LO peak dissapears.")
rich.print(f"5.\tWhen only RSB is visible in the spectrum analyser, save the calibration with the last cell.")

calibrated_values = [0,0,0.5,0] if seq.seq_idx == 0 else calibration.loc[len(calibration)-1][-4:].values.tolist() # storage array
#calibrated_values = [-0.004462799999999999*1000, -0.0097599*1000, 0.9824, -5.3845]
print(lo_frequency)
def set_offset_I(offset_I):
    if qxm.module_type.name == "QRM":
        qxm.out0_offset_path0(offset_I) # qrm have only one output
    else:
        #print(f"{select_out.value}_offset_path0")
        offset_path0 = getattr(qxm, f"{select_out.value}_offset_path0")
        offset_path0(offset_I)

    qxm.arm_sequencer(seq.seq_idx)
    qxm.start_sequencer(seq.seq_idx)
    calibrated_values[0] = offset_I

def set_offset_Q(offset_Q):
    if qxm.module_type.name == "QRM":
        qxm.out0_offset_path1(offset_Q) # qrm have only one output
    else:
        offset_path1 = getattr(qxm, f"{select_out.value}_offset_path1")
        offset_path1(offset_Q)
        
    qxm.arm_sequencer(seq.seq_idx)
    qxm.start_sequencer(seq.seq_idx)
    calibrated_values[1] = offset_Q

def set_gain_ratio(gain_ratio):
    seq.mixer_corr_gain_ratio(gain_ratio)
    qxm.arm_sequencer(seq.seq_idx)
    qxm.start_sequencer(seq.seq_idx)
    calibrated_values[2] = gain_ratio

def set_phase_offset(phase_offset):
    seq.mixer_corr_phase_offset_degree(phase_offset)
    qxm.arm_sequencer(seq.seq_idx)
    qxm.start_sequencer(seq.seq_idx)
    calibrated_values[3] = phase_offset

interact(
    set_offset_I, offset_I=widgets.FloatSlider(min=-25.0, max=25.0, step=0.0001, value=calibrated_values[0], readout_format='.4f', layout=widgets.Layout(width='95%' ))
)
interact(
    set_offset_Q, offset_Q=widgets.FloatSlider(min=-25.0, max=25.0, step=0.0001, value=calibrated_values[1], readout_format='.4f', layout=widgets.Layout(width='95%' ))
)
interact(
    set_gain_ratio,
    gain_ratio=widgets.FloatSlider(min=0.5, max=2.0, step=0.0001, value=calibrated_values[2], readout_format='.4f', layout=widgets.Layout(width='95%' )),
)
interact(
    set_phase_offset,
    phase_offset=widgets.FloatSlider(min=-45.0, max=45.0, step=0.00001, value=calibrated_values[3], readout_format='.4f', layout=widgets.Layout(width='95%' )),
)

1.      Connect complex output 'out0' of cluster 'loki_cluster' module 'module17' to a spectrum analyser.

2.      In the spectrum analyser, set the center frequency to 6.804 GHz

3.      Adjust the gain ratio and phase offset sliders until the LSB is gone.

4.      Adjust the I and Q offsets until the LO peak dissapears.

5.      When only RSB is visible in the spectrum analyser, save the calibration with the last cell.

...

interactive(children=(FloatSlider(value=0.0, description='offset_I', layout=Layout(width='95%'), max=25.0, min…

interactive(children=(FloatSlider(value=0.0, description='offset_Q', layout=Layout(width='95%'), max=25.0, min…

interactive(children=(FloatSlider(value=0.5, description='gain_ratio', layout=Layout(width='95%'), max=2.0, mi…

interactive(children=(FloatSlider(value=0.0, description='phase_offset', layout=Layout(width='95%'), max=45.0,…

<function __main__.set_phase_offset(phase_offset)>

In [202]:
calibration.loc[len(calibration)] = [module, select_out.value, seq.seq_idx, lo_frequency, MY_IF, *tuple(calibrated_values)]
calibration

,module,complex_output,sequencer index,lo_freq (Hz),if (Hz),dc_mixer_offset_I,dc_mixer_offset_Q,mixer_amp_ratio,mixer_phase_error_deg
0,module3,out0,0,4.837500e+09,-100000000.0,-10.1431,-10.6007,0.9751,-24.26763
1,module4,out0,0,5.146000e+09,-100000000.0,-11.4242,-5.0180,0.9805,-17.29633
2,module5,out0,0,4.400250e+09,-100000000.0,-4.6043,2.8827,0.9784,-18.85781
3,module6,out0,0,5.214750e+09,-100000000.0,-12.1181,-0.4186,0.9565,-12.61191
4,module7,out0,0,4.722750e+09,-100000000.0,-11.6910,-7.6339,0.9764,-25.34393
5,module8,out0,0,5.277167e+09,-100000000.0,-8.5414,-2.3489,0.9545,-21.62042
6,module9,out0,0,4.633500e+09,-100000000.0,-20.0189,-8.5023,0.9725,-23.90258
7,module10,out0,0,5.312750e+09,-100000000.0,-8.4079,-6.4061,0.9444,-23.06178
8,module16,out0,0,6.793000e+09,-100000000.0,-7.0733,0.7848,1.0046,9.96941
9,module17,out0,0,6.804000e+09,-100000000.0,-4.9380,1.7005,1.0305,4.20397


In [203]:
calibration.to_csv("mixercorrectionLokeB.csv")

In [204]:
cluster.reset()

In [69]:
Out

{2:     module complex_output  sequencer index  lo_freq (Hz)      if (Hz)  \
 0  module1           out0                0  4.250250e+09 -100000000.0   
 1  module2           out0                0  5.416500e+09 -100000000.0   
 
    dc_mixer_offset_I  dc_mixer_offset_Q  mixer_amp_ratio  \
 0            -7.2516             2.8716           0.9871   
 1            -9.4730            -2.6162           0.9578   
 
    mixer_phase_error_deg  
 0              -21.58270  
 1              -17.96838  ,
 11: {'q16': 4250250000.0,
  'q17': 5416500000.0,
  'q18': 4837500000.0,
  'q19': 5146000000.0,
  'q20': 4400250000.0,
  'q21': 5214750000.0,
  'q22': 4722750000.0,
  'q23': 5277166670.0,
  'q24': 4633500000.0,
  'q25': 5312750000.0},
 17: <function __main__.set_phase_offset(phase_offset)>,
 18: <function __main__.set_phase_offset(phase_offset)>,
 23: {'q16': 4250250000.0,
  'q17': 5416500000.0,
  'q18': 4837500000.0,
  'q19': 5146000000.0,
  'q20': 4400250000.0,
  'q21': 5214750000.0,
  'q22': 472

In [213]:
[print(val) for val in calibration['lo_freq (Hz)'].values]

4837500000.0
5146000000.0
4400250000.0
5214750000.0
4722750000.0
5277166670.0
4633500000.0
5312750000.0
6793000000.0
6804000000.0


[None, None, None, None, None, None, None, None, None, None]

In [215]:
[print(val*1e-3) for val in calibration['dc_mixer_offset_I'].values]

-0.010143099999999999
-0.0114242
-0.0046043
-0.012118099999999998
-0.011691
-0.008541400000000001
-0.0200189
-0.008407899999999998
-0.0070733
-0.004937999999999999


[None, None, None, None, None, None, None, None, None, None]

In [218]:
[print(val*1e-3) for val in calibration['dc_mixer_offset_Q'].values]

-0.0106007
-0.0050180000000000025
0.0028826999999999998
-0.00041860000000000145
-0.007633899999999999
-0.0023489000000000006
-0.008502299999999999
-0.006406099999999999
0.0007848000000000006
0.0017005000000000017


[None, None, None, None, None, None, None, None, None, None]

In [222]:
[print(val) for val in calibration['mixer_amp_ratio'].values]

0.9751000000000001
0.9805000000000001
0.9784
0.9564999999999999
0.9763999999999999
0.9545
0.9725000000000001
0.9444000000000001
1.0046
1.0305


[None, None, None, None, None, None, None, None, None, None]

In [223]:
[print(val) for val in calibration['mixer_phase_error_deg'].values]

-24.26763
-17.296329999999998
-18.85781
-12.611909999999995
-25.343929999999997
-21.62042
-23.902579999999997
-23.06178
9.969409999999996
4.203970000000005


[None, None, None, None, None, None, None, None, None, None]

In [216]:
4.250250e+09

4250250000.0

In [217]:
5.416500e+09

5416500000.0

In [ ]:
module complex_output  sequencer index  lo_freq (Hz)      if (Hz)  \
 0  module1           out0                0  4.250250e+09 -100000000.0   
 1  module2           out0                0  5.416500e+09 -100000000.0   
 
    dc_mixer_offset_I  dc_mixer_offset_Q  mixer_amp_ratio  \
 0            -7.2516             2.8716           0.9871   
 1            -9.4730            -2.6162           0.9578  

     mixer_phase_error_deg  
 0              -21.58270  
 1              -17.96838  ,